# Knapsack

- Fabio Mejia
- Juan Manuel Lopez 
- Johan Cortes

The knapsack problem is a popular computer science problem. Part of the reason for it's popularity is it's combination of simple rules but difficult solution, as it has no solution in polynomial time. This problem is therefore an NP-complete problem, which means no correct solution exists that is also fast (polynomial time).

The game consists of the following:
- A knapsack (backpack) with a maximum capacity W
- N distinct items with a wegiht (Wi) and value (Vi)

Which of these items can you fit in the bag that maximises value (sum of items Vi) but does not exceed the maximum capacity W?

![img](https://upload.wikimedia.org/wikipedia/commons/f/fd/Knapsack.svg)

As an example, say you have items (Vi, Wi) and a knapsack with capacity of 15:
1. (4 \$, 12 kg)
2. (2 \$, 2 kg)
3. (2 \$, 1 kg)
4. (1 \$, 1 kg)
5. (10 \$, 4 kg)

Which combination of these achieves the best result? In this case, the answer is items 2-5, which add up to 15$ in value and 8kg in weight.

There is another variation of this problem, what if we could pick any amount of each item? In this case, we would add three of 5 and three of 3, which add up to 36$ with weight 15kg.

Your task is to write a genetic algorithm that computes the solution to the knapsack problem.

In [16]:
import random
import copy
import numpy as np

backpack_threshold = 15

class Item:
  def __init__(self, value, weight, quantity):
    self.weight = weight
    self.value = value
    self.quantity = quantity
  
  def __str__(self):
    return "%s, %s, %s" % (self.value, self.weight, self.quantity)

  def __repr__(self):
    return "item: %s, %s, %s" % (self.value, self.weight, self.quantity)

In [17]:
# Resive una lista de items para definirles un valor de fitness 
# si el peso de los items en la maleta sobrepasa la capcidad el fitness es -1
def evaluate(items):
  sum_w = 0
  sum_v = 0

  for item in items:
    sum_w += item.weight*item.quantity
    sum_v += item.value*item.quantity
  
  if sum_w > backpack_threshold:
    return -1

  return sum_v


In [18]:
# Resive una generacion y  crea una tupla de una lista de items y su puntuacion de fitness
# 
def evaluate_many(generation):
  return [ (individual, evaluate(individual)) for individual in generation ]

evaluate_many([[Item(4, 12, 1), Item(2, 2, 0)], [Item(4, 12, 0), Item(2, 2, 1)], [Item(4, 12, 1), Item(2, 2, 1)]])

[([item: 4, 12, 1, item: 2, 2, 0], 4),
 ([item: 4, 12, 0, item: 2, 2, 1], 2),
 ([item: 4, 12, 1, item: 2, 2, 1], 6)]

In [19]:
# Resive lista tuplas con lista de items y su puntuacion de fitness
# Ordena a lista segun el puntaje de cada tupla y saca n cantidades del tope 
def selection(evaluated_generation, top_n):
  evaluated_generation.sort(key = lambda e: e[1], reverse = True)
  return evaluated_generation[0:top_n]

selection([
  ([Item(4, 12, 0), Item(2, 2, 0)], 0),
  ([Item(4, 12, 1), Item(2, 2, 0)], 4),
  ([Item(4, 12, 0), Item(2, 2, 1)], 2)
], 2)

[([item: 4, 12, 1, item: 2, 2, 0], 4), ([item: 4, 12, 0, item: 2, 2, 1], 2)]

In [20]:
# Resive una  lista de items  y un n de mutaciones que dice la variacion que se aplica a un item
# devuelve una lista de items que mutados
def mutate(offsprings, n_mutations):
  new_offsprings = []
  for offspring in offsprings:
    delta = random.randint(-n_mutations, n_mutations)
    new_offsprings.append(Item(offspring.value, offspring.weight, max(0, offspring.quantity+delta)))
  
  return new_offsprings

In [21]:
# Resive una lista de los mejores casos, la cantidad de la nueva generacion mutada y la cantidad de genes a mutar
# Devuelve una generacion mutada
def mutate_many(best_individuals, generation_size, n_genes):
  new_generation = []

  for i in range(generation_size):
    new_individual = mutate(random.choice(best_individuals), n_genes)
    new_generation.append(new_individual)

  return new_generation

mutate_many([[Item(4, 12, 0), Item(2, 2, 0)], [Item(4, 12, 0), Item(2, 2, 0)]], 10, 1)

[[item: 4, 12, 0, item: 2, 2, 0],
 [item: 4, 12, 0, item: 2, 2, 0],
 [item: 4, 12, 0, item: 2, 2, 0],
 [item: 4, 12, 0, item: 2, 2, 1],
 [item: 4, 12, 1, item: 2, 2, 0],
 [item: 4, 12, 0, item: 2, 2, 0],
 [item: 4, 12, 1, item: 2, 2, 0],
 [item: 4, 12, 0, item: 2, 2, 0],
 [item: 4, 12, 0, item: 2, 2, 1],
 [item: 4, 12, 0, item: 2, 2, 1]]

In [22]:
MAX_GENERATIONS = 20
TOP_N = 4
MUTATED_CHROMOSOMES = 1
GENERATION_SIZE = 30
FITNESS_THRESHOLD = 50
MAXIMUM_WEIGHT = 15
ITEMS = [Item(4, 12, 0), Item(2, 2, 0), Item(2, 1, 0), Item(1, 1, 0), Item(10, 4, 0)]

n_generation = 0
converged = False

generation = [[copy.deepcopy(item) for item in ITEMS] for _ in range(GENERATION_SIZE)]

for i in range(GENERATION_SIZE):
  for j in range(len(generation[i])):
    c = random.randint(0,1)
    generation[i][j].quantity = c

weights = np.array([item.weight for item in ITEMS])
values = np.array([item.value for item in ITEMS])



while n_generation < MAX_GENERATIONS and not converged:
  evaluated_generation = evaluate_many(generation)
  evaluated_best = selection(evaluated_generation, TOP_N)
  # print(evaluated_best)
  parents = [item for item,fitness in evaluated_best]
  
  best_individuals = [ individual for individual, evaluation in evaluated_best ]
  best_individuals = best_individuals[:GENERATION_SIZE]

  generation = generation + mutate_many(best_individuals, GENERATION_SIZE, MUTATED_CHROMOSOMES)
  
  best_in_generation = evaluated_best[0]
  converged = best_in_generation[1] >= FITNESS_THRESHOLD
  n_generation += 1

  
  print(f"{n_generation}: {best_in_generation}")

1: ([item: 4, 12, 0, item: 2, 2, 1, item: 2, 1, 1, item: 1, 1, 0, item: 10, 4, 1], 14)
2: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 2, item: 1, 1, 0, item: 10, 4, 2], 24)
3: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 2, item: 1, 1, 0, item: 10, 4, 3], 34)
4: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 2, item: 1, 1, 0, item: 10, 4, 3], 34)
5: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
6: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
7: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
8: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
9: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
10: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
11: ([item: 4, 12, 0, item: 2, 2, 0, item: 2, 1, 3, item: 1, 1, 0, item: 10, 4, 3], 36)
12: ([item: 4, 12, 0, item: 2, 2, 0, item